In [ ]:
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
import dataframe_image as dfi

from Project1 import *

import plotly.io as pio
pio.renderers.default = 'iframe'

# Boston Airbnb Dataset
https://www.kaggle.com/datasets/airbnb/boston?resource=download

CRISP-DM:
- Business understanding: cusomters of Airbnb - hosts and guests - ultimately need to converge on price. What factors influence prices?
    - **Price** is our dependent variable.
    - We'll explore the dataset to determine our **independent variables** (predictors for a model).
- Data understanding: Given raw data from Kaggle, what variables in the dataset can be used to predict price?
- Data preparation: evaluate the data by type, look for what can be used for a parsimonious model, fill missing data where possible, exclude data where justifiable.
    - Use visualizations, aggregation, filtering to better understand and prepare the data.
- Data modeling: select an appropriate algorithm to model price given selected predcitors.
    - Algo should be robust to missing values, and not senstive to potential correlation among the independent variables.
    - Algo should be able to utilize both numeric and categorical values among the independent variables.
- Result evaluation: train a model and evaluate the fit.
- Deployment: predict prices using test (out of sample) data and summarize findings in a post on Medium.

Three questions/topics to explore:
- How does **geography** inlfuence Airbnb **prices** in Boston?
    - What areas/zip codes/neighborhoods are more expensive than others?
- How do **property characteristics** influence Airbnb prices in Boston?
    - Number of bathrooms, bedrooms, beds, square footage, reviews, etc.
- How does **time (seasonality)** influence Airbnb prices in Boston?
    - Period of the week (days), period of the month (weeks), period of the year (months)?
    - We only have about 2 years of data to work with, but we'll see what comes out.

# First, load and clean the data
- I did not ultimately use *monthly_price*/*weekly_price* in the model, but I used OLS to estimate the missing values based on *price*.
- I used a KNN classifier to fill in missing zip codes.

In [ ]:
boston_listings = pd.read_csv('./data/airbnb_boston/listings.csv', index_col='id')
boston_calendar = pd.read_csv('./data/airbnb_boston/calendar.csv')
boston_reviews = pd.read_csv('./data/airbnb_boston/reviews.csv')

boston_listings = boston_listings.map(convert_dollars_to_float).map(convert_percentages_to_float).map(convert_string_date_to_dt)
boston_calendar = boston_calendar.apply(convert_dollars_to_float).map(convert_percentages_to_float).map(convert_string_date_to_dt)
boston_reviews = boston_reviews.apply(convert_dollars_to_float).map(convert_percentages_to_float).map(convert_string_date_to_dt)

# estimate missing monthly_price/weekly_price fields using regression (based on price field)
boston_listings = estimate_y_from_X_ols(
    data=boston_listings, y_label='monthly_price', X_labels='price')['filled_data']
boston_listings = estimate_y_from_X_ols(
    data=boston_listings, y_label='weekly_price', X_labels='price')['filled_data']

# estimate missing zip codes using KNN classification (based on latitude and longitude)
boston_listings = classify_y_based_on_X_knn(
    data=get_cleaned_zipcodes(boston_listings), y_label='zipcode_cleaned', X_labels=['latitude', 'longitude'])['filled_data']

In [ ]:
# identify column types
boston_listing_col_types = get_columns_and_types(boston_listings)
boston_calendar_col_types = get_columns_and_types(boston_calendar)
boston_review_col_types = get_columns_and_types(boston_reviews)

# Are there outliers in *price*?

In [ ]:
outlier_cutoff = 500
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))
boston_listings['price'].plot.hist(
    title='Distribution of All Prices', ax=axes[0]);
boston_listings['price'].sort_values(ascending=False).head(100).plot.hist(
    title='Distribution of Largest 100 Prices', ax=axes[1]);
boston_listings['price'].where(lambda x: x <= outlier_cutoff).dropna().plot.hist(
    title=f'Distribution of Prices\nLess Than/Equal to ${outlier_cutoff}', ax=axes[2]);
plt.savefig('price_histograms.png')

# Question 1: How does geography influence Airbnb rental prices?

## Add a choropleth (geographical map) and overlay the listings prices from the dataset.
- Thank you for the geojson data: https://github.com/codeforgermany/click_that_hood/blob/main/public/data/boston.geojson?short_path=46589b4
- Use log scale for the prices (so the colorbar isn't too compressed) - or change the scale of the colorbar.

A quick visual inspection of the map allow us to see more expensive listings tend to northward, and in the following neighborhoods (in no particular order):
- West End, North End, South End, Downtown, Leather District, Chinatown, Leather District, South Boston Waterfront, Fenway.

- What if we run a regression-type model of price on latitude+longitude?

In [ ]:
# geoplot of neighborhoods with listings overlaid
with open('./data/airbnb_boston/boston.geojson', 'r') as f:
    geojson = json.load(f)
    
geoplot_data = boston_listings[['latitude', 'longitude', 'zipcode_cleaned', 'price']].dropna()
geoplot_data['log_price'] = geoplot_data['price'].apply(np.log)

fig = px.choropleth(
    data_frame={'name': [i['properties']['name'] for i in geojson['features']]}, 
    geojson=geojson, 
    locations='name', 
    featureidkey="properties.name",
    title='Boston Neighborhoods and Airbnb Prices<br>Colorbar is Log Scale',
)
fig.update_geos(fitbounds="locations", visible=False) 
fig.add_trace(
    px.scatter_geo(
        data_frame=geoplot_data, 
        lat='latitude', 
        lon='longitude', 
        color='log_price',
        hover_data={
            'latitude': ':.2f', 
            'longitude': ':.2f', 
            'price': ':.2f', 
            'log_price': ':.2f',
        },
    ).data[0])

# relabel the colorbar (as showing log values is confusing)
fig.update_coloraxes(colorbar={
    'title': 'Price',
    'tickvals': geoplot_data['log_price'].quantile([0.01, 0.999]).values,
    'ticktext': ['Cheaper', 'Pricier'],
})
fig.update_layout(showlegend=False)

fig.show()

pio.write_image(fig, 'prices_and_neighborhoods.png') 

In [ ]:
prices_by_neighborhood = pd.concat([
    boston_listings[['price', 'neighbourhood_cleansed']].groupby('neighbourhood_cleansed').mean().squeeze().to_frame('Mean Price'),
    boston_listings[['price', 'neighbourhood_cleansed']].groupby('neighbourhood_cleansed').median().squeeze().to_frame('Median Price'),
], axis=1).sort_values(by='Median Price', ascending=False)
prices_by_neighborhood_styled = prices_by_neighborhood.style.format('${:.0f}').set_caption('Mean and Median Price by Neighborhood<br>Sorted by Median')
dfi.export(prices_by_neighborhood_styled, 'prices_by_neighborhood_table.png')
prices_by_neighborhood_styled

# What percentage of observations are below the outlier cutoff (say, below $500)?

In [ ]:
f'Percent of observations more than ${outlier_cutoff}: {
    len(boston_listings['price'].where(lambda x: x > outlier_cutoff).dropna())/len(boston_listings):.1%
    }'

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax = prices_by_neighborhood['Median Price'].plot.bar(cmap='viridis', title='Sorted Bar Plot of Median Price by Neighborhood')
ax.axvline(11.5, color='black', alpha=0.2)
ax.axhline(150, color='black', alpha=0.2)
fig.tight_layout()
plt.savefig('prices_by_prices_by_neighborhood_barplot.png')

# Let's look at how well a regression can fit price on latitude and longitude
- This gives a very poor fit, even on the training set. Why?
    - It could be that latitude and longtide - which reflect locations on a sphere (the Earth) do not align well with 2d locations on a map.
    - Therefore, the link betweeb price and lat/lon may not be linear, which means regression is not the right tool.
    - Let's keep exploring.

In [ ]:
reg_price_on_lat_lon = estimate_y_from_X_ols(
    data=boston_listings,
    y_label='price',
    X_labels=['latitude', 'longitude'],
    train_size=0.6,
    random_state=42,
    add_constant=False,
)

# Question 2: How do characteristics influence Airbnb listing prices?

In [ ]:
categorical_vars_to_use = [
    'bathrooms',               # number of bathrooms
    'bedrooms',                # number of bedrooms
    'beds',                    # number of beds
    'accommodates',            # number of occupants that can be accomodated 
    'property_type',           # House, Apartment, Condoninium, etc
    'room_type',               # Entire home/apt, Private room, Shared room
    'cancellation_policy',     # moderate, flexible, strict, super_strict_30
    'host_identity_verified',  # f, t (boolean)
    'neighbourhood_cleansed',  # cleaned neighborhood label
]

numeric_features_to_use = [
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'zipcode_cleaned'
]

grid = sns.pairplot(
    data=boston_listings[['price'] + categorical_vars_to_use + numeric_features_to_use].dropna(),
)
grid.fig.suptitle('Pairplot of Price, Categorical Variables and Numerica Variables');
grid.fig.tight_layout()
grid.fig.savefig('variable_pairplot.png')


In [ ]:
variable_nan_details = pd.concat({
    'NaN Count': boston_listings[['price'] + categorical_vars_to_use + numeric_features_to_use].isnull().sum(),
    'NaN %': boston_listings[['price'] + categorical_vars_to_use + numeric_features_to_use].isnull().sum()/len(boston_listings),
}, axis=1).rename_axis('Feature', axis=0).style.format('{:.1%}', subset='NaN %').set_caption('Missing Value Details for Selected Features')
display(variable_nan_details)

dfi.export(variable_nan_details, 'variable_nan_details.png')

# Question 3: How does time influence Airbnb listing prices?
- Day of week.
- Week of month.
- Month of year.

In [ ]:
# convert price column to a float, assuming format of $#.#
boston_calendar.price = [float(i.replace('$', '').replace(',', '')) if type(i) is str else i for i in boston_calendar.price]

# concert date to a datetime object
boston_calendar.date = [pd.Timestamp(i) if type(i) is str else i for i in boston_calendar.date]

# add a month, year columns for seasonality analysis
day_map = {0: 'Mon', 1: 'Tues', 2: 'Wed', 3: 'Thurs', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
boston_calendar['weekday'] = [i.weekday() for i in boston_calendar.date]
boston_calendar['month'] = [i.month for i in boston_calendar.date]
boston_calendar['year'] = [i.year for i in boston_calendar.date]

In [ ]:
boxplot_data = boston_calendar.assign(log_price=lambda x: np.log(x['price'])).dropna()
ax = sns.boxplot(
    data=boxplot_data,
    x='month',
    y='log_price',
    showfliers=False,
    palette={month: 'salmon' if month in [9, 10, 11] else 'dodgerblue' for month in boxplot_data.month.unique()},
    hue='month', legend=False,
)
ax.set_title("Distribution of Log Listing Prices by Calendar Month");
plt.savefig('price_by_month_boxplot.png')

In [ ]:
ax = sns.boxplot(
    data=boxplot_data,
    x='weekday',
    y='log_price',
    showfliers=False,
    palette={weekday: 'salmon' if weekday in [4, 5] else 'dodgerblue' for weekday in boxplot_data.weekday.unique()},
    hue='weekday', legend=False,
)
ax.set_title("Distribution of Log Listing Prices by Day of the Week");
ax.set_xticks(ax.get_xticks());
ax.set_xticklabels(day_map[i] for i in ax.get_xticks());
plt.savefig('price_by_weekday_boxplot.png')

What other numerical values can we use (and what needs to be filled before we can)?
- host_response_rate (missing around 400 entries)
- host_acceptance_rate (missing around 400 entries)
- square_feet (missing a lot)
- extra people?
- cleaning fee?
- security_deposit?

# Assemble a model
- Us a random forest regressor. This can handle both categorical and numeric values, and also can handle nans (especially in the numeric values, like the reviews, which contain numerous nans).

In [ ]:
y_label = 'price'
outlier_ids = list(boston_listings['price'].where(lambda x: x > outlier_cutoff).dropna().index)

numeric_features_to_use = [
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'zipcode_cleaned',
]

categorical_vars_to_use = [
    'bathrooms',               # number of bathrooms
    'bedrooms',                # number of bedrooms
    'beds',                    # number of beds
    'accommodates',            # number of occupants that can be accomodated 
    'property_type',           # House, Apartment, Condoninium, etc
    'room_type',               # Entire home/apt, Private room, Shared room
    'cancellation_policy',     # moderate, flexible, strict, super_strict_30
    'host_identity_verified',  # f, t (boolean)
    'neighbourhood_cleansed',  # cleaned neighborhood label
]

model_data = pd.concat([
    boston_listings[[y_label]],
    pd.get_dummies(boston_listings[categorical_vars_to_use], columns=categorical_vars_to_use, drop_first=True),
    boston_listings[numeric_features_to_use],
    ], axis=1).drop(outlier_ids)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    model_data.drop(y_label, axis=1), 
    model_data[y_label],
    train_size=0.6, 
    random_state=42,
)

m_rf = RandomForestRegressor(
    random_state=42,
)
m_rf.fit(X_train, y_train);

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

scatter_data_train = pd.concat({
        'Y_HAT': pd.Series(m_rf.predict(X_train)),
        'Y_OBS': y_train.reset_index(drop=True),
    }, axis=1)
scatter_data_test = pd.concat({
        'Y_HAT': pd.Series(m_rf.predict(X_test)),
        'Y_OBS': y_test.reset_index(drop=True),
    }, axis=1)

sns.regplot(
    data=scatter_data_train,
    y='Y_HAT',
    x='Y_OBS',
    ax=axes[0],
    ci=None,
    line_kws={'color': 'dodgerblue'},
)
axes[0].set_title(f'Random Forest\nTraining Data, Score: {m_rf.score(X_train, y_train):.2f}')

sns.regplot(
    data=scatter_data_test,
    y='Y_HAT',
    x='Y_OBS',
    ax=axes[1],
    ci=None,
    line_kws={'color': 'dodgerblue'},
)
axes[1].set_title(f'Random Forest\nTesting Data: Score: {m_rf.score(X_test, y_test):.2f}')
fig.tight_layout()

ymin = pd.concat([scatter_data_test, scatter_data_train], axis=0).min().min()
ymax = pd.concat([scatter_data_test, scatter_data_train], axis=0).max().max()
for i in axes:
    ax.set_ylim((ymin, ymax))

plt.savefig('model_train_test.png')
